In [20]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [25]:
data_path = r'C:\UCSC-Sam\numpy_project\Astr19\ASTR19_F23_group_project_data (1).txt'
days = []
times = []
heights = []

with open(data_path, 'r') as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) == 3:
            day, time_str, height = parts
            try:
                # Convert day to int, height to float, and time to decimal hours
                day = int(day)
                hour, minute = map(int, time_str.split(':'))
                decimal_time = hour + minute / 60.0
                height = float(height)
                
                # Store in lists
                days.append(day)
                times.append(decimal_time)
                heights.append(height)
            except ValueError as e:
                print(f"Skipping line due to error: {e}")

# Convert lists to NumPy arrays for analysis
days = np.array(days)
times = np.array(times)
heights = np.array(heights)
continuous_time = days * 24 + times  # Create a continuous time variable



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/ASTR19_F23_group_project_data (1).txt'

In [23]:
# Define the sine wave model
def sine_model(time, amplitude, frequency, phase, offset):
    return amplitude * np.sin(frequency * time + phase) + offset

# Initial parameter guesses
initial_guess = [5, 0.01, 0, 0]  # [Amplitude, Frequency, Phase, Offset]

# Perform the curve fitting
params, params_cov = curve_fit(sine_model, continuous_time, heights, p0=initial_guess)

In [24]:
model_time = np.linspace(continuous_time.min(), continuous_time.max(), 1000)
model_height = sine_model(model_time, *params)

plt.figure(figsize=(10, 6))
plt.scatter(continuous_time, heights, label='Observed Data')
plt.plot(model_time, model_height, color='red', label='Fitted Model')
plt.xlabel('Continuous Time (hours)')
plt.ylabel('Tide Height (ft)')
plt.legend()
plt.title('Tide Height: Data vs. Model')
# plt.savefig('/mnt/data/tide_model_fit.pdf')
plt.show()

AttributeError: 'list' object has no attribute 'min'